In [1]:
import glob
import codecs
from utils import in_blocklist

blocklist = []
blocklist.append('AWAYLEN=307 MAXTARGETS=20 WALLCHOPS WATCH=128 WATCHOPTS=A SILENCE=15 MODES=12 CHANTYPES=# PREFIX')
blocklist.append('- * -')
blocklist.append('XxXChatters.Com')
blocklist.append('XxXChatters')
blocklist.append('This server was created')
blocklist.append('operator(s) online')
blocklist.append('is now your displayed host')
blocklist.append('Caps set:')
blocklist.append('MAXCHANNELS')
blocklist.append('http')
blocklist.append('type it in the main room or a pm to the')
blocklist.append('The topic is')
blocklist.append('Topic set by')

all_separated = []
all_posts = []
all_posts_410 = []
all_posts_501 = []
authors = {}
authors_410 = {}
authors_501 = {}

def parse_all(files, separate):
    for file in glob.glob(files):
        lines = codecs.open(file, mode='r', encoding='utf-8')

        for line in lines:
            if not in_blocklist(blocklist, line.strip()): # blocked?
                if line[0] is '[' and line.find("<") is not -1:
                    s = line
                    sentence = s[s.find(">")+2:]
                    time = s[s.find("[")+1:s.find("]")]
                    author = s[s.find("<")+1:s.find(">")]
                    
                    p = (time+' ' + author, sentence.strip().lower())
                    all_posts.append(p)
                    if separate in file:
                        all_separated.append(p)

                    if author not in authors: # save things
                        authors[author] = set() # init list for author
                    authors[author].add(sentence.strip().lower()) 
                else:
                    if line.find('*') is not -1 and ") has joined #" not in line and ") Quit (" not in line and ") has left #" not in line and "6A14thena14, 06y14o14u 06n14eve14r" not in line and " sets mode: +" not in line and " is now known as " not in line:
                        time = line[line.find("[")+1:line.find("]")]
                        author_beginning = line.find("* ")
                        aa = line[author_beginning+2:]
                        aa_end = author_beginning + 2 + aa.find(" ")
                        author = line[author_beginning+2:aa_end]
                        sentence = line[aa_end + 1:]
                        
                        p = (time+' ' + author, sentence.strip().lower())
                        all_posts.append(p)
                        if separate in file:
                            all_separated.append(p)

                        if author not in authors: # save things
                            authors[author] = set() # init list for author
                        authors[author].add(sentence.strip().lower()) 
                    if ") has joined #" in line or ") Quit (" in line or ") has left #" in line:
                        time = line[line.find("[")+1:line.find("]")]
                        author_beginning = line.find("* ")
                        aa = line[author_beginning+2:]
                        aa_end = author_beginning + 2 + aa.find(" ")
                        author = line[author_beginning+2:aa_end]
                        if ") has joined #" in line:
                            author += " JOINED THE ROOM"
                        else:
                            author += " LEFT THE ROOM"
                        sentence = ""
                        p = (time+' ' + author, sentence)
                        all_posts.append(p)
                        if separate in file:
                            all_separated.append(p)
                            
                        if author not in authors: # save things
                            authors[author] = set() # init list for author
                        authors[author].add(sentence) 

    for line in codecs.open('1006/410_utf8.txt', mode='r', encoding='utf-8'):
        if ':' in line and not in_blocklist(blocklist, line.strip()): # blocked?:
            sentence = line[line.find(":"):]
            all_posts.append((author, sentence.strip().lower()))
            all_posts_410.append((author, sentence.strip().lower()))

            author = line[:line.find(":")]
            if author not in authors: # save things
                authors[author] = set() # init list for author
            if author not in authors_410:
                authors_410[author] = set()
            authors[author].add(sentence.strip().lower())
            authors_410[author].add(sentence.strip().lower())


    for s in codecs.open("1006/501_utf8.txt", mode='r', encoding='utf-8').readlines():
        if ':' in s and 'has left the channel' not in s and 'has quit' not in s and 'joined the channel' not in s:
            if not in_blocklist(blocklist, s.strip()): # blocked?
                sentence = s[s.find(">")+2:]
                all_posts.append((author, sentence.strip().lower()))
                all_posts_501.append((author, sentence.strip().lower()))

                author = s[s.find("<")+1:s.find(">")]
                if author not in authors: # save things
                    authors[author] = set() # init list for author
                if author not in authors_501:
                    authors_501[author] = set()
                authors[author].add(sentence.strip().lower())
                authors_501[author].add(sentence.strip().lower()) 
        
parse_all("xxxchatters_logs/*.log", "#chat")
#for file in glob.glob("xxxchatters_logs/*.log"):
print(len(all_posts))
print(len(all_separated))
print(len(authors))
print(len(authors_410))
print(len(authors_501))
print(len(all_posts_410))
print(len(all_posts_501))

479404
158959
29086
40
73
304
238


In [3]:
for a in authors['carrol']:
    print(a)


In [35]:
import operator
import string
word_counts = dict()
translator = str.maketrans('', '', string.punctuation)

def preprocess_word(word):
    word = word.replace('\x02', '')
    word = word.replace('\x03', '')
    word = word.replace('.', '')
    word = ''.join([i for i in word if not i.isdigit()])

    #word = word.translate(translator)
    word = ''.join(e for e in word if e.isalnum())
    return word


for post in all_posts:
    word_list = post[1].split()
    for word in word_list:
        word = preprocess_word(word)
        #word = word.replace('\x03', '')
        #word = word.replace('.', '')

        #word = word.translate(translator)
        #word = ''.join(e for e in word if e.isalnum())
        if len(word) < 1:
            continue
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

sorted_word_counts = sorted(word_counts.items(), key=operator.itemgetter(1), reverse=True)

def get_sorted_word_counts():
    return sorted_word_counts

v = get_sorted_word_counts()
print(len(v))

49231


In [36]:
def get_topn_percent(perc):
    counts = get_sorted_word_counts()
    l = len(counts)
    max_index = int(perc*l)
    top_n = []
    #print(counts[:max_index])
    for t in counts[:max_index]:
        top_n.append(t[0])
    return top_n

def get_all_by_max_appearance(app):
    counts = get_sorted_word_counts()
    processed_counts = []
    for c in counts:
        if c[1] >= app:
            processed_counts.append(c[0])
        
    return processed_counts

def get_by_appearance_list(appearance):
    # appearance = [2, 3, 4] # number indicating times of appearance
    counts = get_sorted_word_counts()
    processed_counts = []
    for c in counts:
        if c[1] in appearance:
            processed_counts.append(c[0])
        
    return processed_counts
    
#topn = get_topn_percent(0.001)

selected_appearances = get_by_appearance_list([110, 112])
print(selected_appearances)

#maxapp = get_all_by_max_appearance(2)
#file = open("1106/chat_room_word_appearance.txt", 'w')
#for e in maxapp:
#    try:
#        file.write(e[0] + ", " + str(e[1]) + '\n')
#    except:
#        pass
#file.close()

['extra', 'arm', 'large', 'past', 'quickly', 'irc', 'human', 'rl', 'size', 'wine', 'leaves', 'firm', 'able', 'summer', 'dylan', 'devil', 'naderaax', 'kinky', 'jag']


In [37]:
def remove_non_frequent(allowed_words, post, replace=True):
    good_words = []
    for word in post.lower().split():
        pword = preprocess_word(word)
        if pword in allowed_words:
            good_words.append(pword)
        else:
            if replace:
                replacement = ""
                for char in word:
                    replacement += "-"
                good_words.append(replacement)
    
    return " ".join(good_words)


def remove_by_probability(allowed_words, always_allowed, post, chance, replace=True):
    import random
    good_words = []
    for word in post.lower().split():
        #pword = preprocess_word(word)
        if word in always_allowed:
            good_words.append(word)
            continue
            
        if word in allowed_words:
            good_words.append(word)
        else:
            if replace:
                rv = random.uniform(0.0, 1.0)
                if rv < chance:
                    replacement = ""
                    for char in word:
                        replacement += "-"
                    good_words.append(replacement)
                else:
                    good_words.append(word)
    
    return " ".join(good_words)

sentence_to_test = "Jus to keep being evil<<<< wops Athena on the backside with a new nerf bat"
print(remove_non_frequent(get_topn_percent(0.1), sentence_to_test))

print(remove_by_probability(get_topn_percent(0.1), get_topn_percent(0.01), sentence_to_test, 0.5))

--- to keep being evil ---- athena on the -------- with a new nerf bat
--- to keep being evil<<<< ---- athena on the -------- with a new nerf bat


In [5]:
sentence_to_test = "bops saffron on the backside with one of my new nerf bats"
print(remove_non_frequent(get_topn_percent(0.01), sentence_to_test))
print(remove_non_frequent(get_topn_percent(0.1), sentence_to_test))


---- ------- on the -------- with one of my new ---- ----
bops saffron on the -------- with one of my new nerf bats


In [38]:
import numpy as np

def get_by_probability(authors_list, count, topn = 30):
    author_probability = {}
    
    for authors in authors_list:
        for key, value in authors.items():
            if key not in author_probability:
                author_probability[key] = len(value)
            else:
                author_probability[key] += len(value)
                
    author_probability_counts = sorted(author_probability.items(), key=operator.itemgetter(1), reverse=True)
    
    #print(author_probability_counts[:topn])

    sum_sentences = 0
    for v in author_probability_counts[:topn]:
        sum_sentences += v[1]

    author_probability_final = {}
    author_list = []
    prob_list = []

    for v in author_probability_counts[:topn]:
        author_probability_final[v[0]] = v[1]/float(sum_sentences)
        author_list.append(v[0])
        prob_list.append(v[1]/float(sum_sentences))

    final_autor_list = np.random.choice(
      author_list, 
      count,
      p=prob_list
    )

    return final_autor_list

def get_topn_authors(author_lists, n):
    author_probability = {}
    
    for author_list in author_lists:
        for key, value in author_list.items():
            if key not in author_probability:
                author_probability[key] = len(value)
            else:
                author_probability[key] += len(value)

    author_probability_counts = sorted(author_probability.items(), key=operator.itemgetter(1), reverse=True)
    
    return author_probability_counts[:n]

#print(get_by_probability(authors, 20))
#print(get_by_probability(authors_410, 20))
#print(get_by_probability(authors_501, 20))

print(get_topn_authors([authors_410,authors_501], 30))
#print(get_by_probability([authors_410,authors_501], 200))

[('@erotic_kitty', 65), ('carrol', 53), ('+DJ`Mercury', 36), ('Louise', 28), ('@erotic_kitty{S}', 27), ('Valkyrie', 21), ('Threeleggedcat', 20), ('PlayfulBBC', 18), ('@saffron{WH}', 17), ('WhoGiveSaDamn', 15), ('Silver_haired`Fox', 14), ('sweet_teresa', 10), ('Woman', 8), ('Jothom', 8), ('Cruel`Intentions', 7), ('guynextdoor', 7), ('Anastatia', 7), ('^fran', 7), ('Sanger', 7), ('Jaems', 6), ('+DJ`South', 6), ('JFetish', 5), ('FunkyBoogieKing', 5), ('gracie', 4), ('@DJ`liltech', 4), ('Handyman', 4), ('MeanMark1', 4), ('Athena', 3), ('saffron{WH}', 3), ('TricksyM', 3)]


In [16]:
# load osc config and parse for later use
import csv

def load_osc_config(file_name):
    config = {}
    with open(file_name, 'r') as csv_file:
        reader = csv.reader(csv_file, delimiter='\t', quotechar='|')
        index = 0
        for row in reader:
            if index > 0:
                #print(row)
                for i in range(len(row)):
                    config[i].append(row[i])
            else:
                config[0] = []
                config[1] = []
                config[2] = []
                config[3] = []
                config[4] = []
            index += 1
    
    return config
            
osc_config = load_osc_config('osc_config.csv')

In [19]:
import random
import sys

def generate_lines_linear(count):
    return_lines = []
    while len(return_lines) < count:
#    for i in range(1, count):
        sentence = random.choice(all_posts)[1]
        perc = len(return_lines)/float(count)

        topn = get_topn_percent(perc)
        #print(perc)
        #print(len(topn))
        #print(sentence)
        stripped_post = remove_non_frequent(topn, sentence.strip().lower())
        if len(stripped_post) > 20 and stripped_post not in return_lines:
            return_lines.append(stripped_post)
        #print(str(perc) + ": " + stripped_post)
        
    return return_lines
        
def generate_lines_from(fromp, step, count):
    for i in range(count):
        sentence = random.choice(all_posts)
        perc = fromp + i * step

        topn = get_topn_percent(perc)
        #print(len(topn))
        stripped_post = remove_non_frequent(topn, sentence[1].strip().lower())
        print("{:.6f}".format(perc) + ": " + stripped_post)
        
def generate_by_osc(config, lists, offset=90):
    assert(len(config) is not len(lists), "len of config and lists needs to be the same")

    index = 0
    possible_words = []
    # length of generated sentences is determined by osc/curve data
    length = len(config[0])
    for l in lists:
        _config = config[index]
        _min = l[0]
        _max = l[1]
        app_list = []
        for i in range(_min, _max+1):
            app_list.append(i)
        
        # all allowed words
        selected_appearances = get_by_appearance_list(app_list)
        possible_words.append((selected_appearances, _config))
        
        index += 1
    
    # all words with min appearance of 500 are always allowed
    always_allowed = get_all_by_max_appearance(500)
        
    finished_sentences = []
    for i in range(length):
        #s = random.choice(all_posts)[1]
        s = all_separated[offset+i][1]
        a = all_separated[offset+i][0]
        # don't process djs
        if 'DJ`Mercury' in a or 'DJ`Protea' in a:
            finished_sentences.append((a, s))
            continue
            
        for p in possible_words:
            _words = p[0]
            probability = float(p[1][i])
            s = remove_by_probability(_words, always_allowed, s, probability)
        
        finished_sentences.append((a, s))
    
    return finished_sentences

def render_plain(out_filename, line_count, offset):
    outfile = codecs.open(out_filename, mode='w', encoding='utf-8')
    for i in range(line_count):
        s = all_separated[offset+i][1]
        a = all_separated[offset+i][0]
        outfile.write(a + ': ' + s + '\n')
    outfile.close()

def render_by_osc(osc_config, out_filename, offset):
    lists = []
    lists.append((1, 5))
    lists.append((6, 20))
    lists.append((21, 50))
    #lists.append((51, 100))
    #lists.append((101, 500))
    osc_config = load_osc_config(osc_config)
    generated = generate_by_osc(osc_config, lists, offset)

    outfile = codecs.open(out_filename, mode='w', encoding='utf-8')
    for line in generated:
        outfile.write(line[0] + ': ' + line[1] + '\n')
    outfile.close()

    

<ipython-input-19-47a6c1561ee5>:33: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(config) is not len(lists), "len of config and lists needs to be the same")


In [21]:
auth = get_by_probability([authors_410,authors_501], 300)
print(auth)

['@erotic_kitty' '@erotic_kitty' '@DJ`liltech']


In [40]:
# samples random lines from corpus, add a author from act1&2 by distribution and outout after a certain length of the line
# randomly add entrences and exits from the last 10 of the top active 30 authors of act1&2

#random.seed(1)
from progressbar import ProgressBar, Bar, Percentage

max_count = 130000
auth = get_by_probability([authors_410,authors_501], max_count)

def generate_lines_maxapp(count, maxapp):
    return_lines = []
    while len(return_lines) < count:
#    for i in range(1, count):
        #perc = len(return_lines)/float(count)
        #perc = 0.5
        sentence = random.choice(all_posts)[1]
        if 'radio meltdown' in sentence:
            continue
        #topn = get_all_by_max_appearance(maxapp)
        topn = get_topn_percent(maxapp)
        stripped_post = remove_non_frequent(topn, sentence.strip().lower())
        #if len(stripped_post) > 20 and len(stripped_post) < 50 and stripped_post not in return_lines:
        if stripped_post not in return_lines:
            return_lines.append(stripped_post)
        #print(str(perc) + ": " + stripped_post)
        
    return return_lines

def process_all(maxapp):
    return_lines = []
    for p in all_posts:
#    for i in range(1, count):
        #perc = len(return_lines)/float(count)
        #perc = 0.5
        sentence = p[1]
        #sentence = random.choice(all_posts)[1]
        if 'radio meltdown' in sentence:
            continue
        #topn = get_all_by_max_appearance(maxapp)
        topn = get_topn_percent(maxapp)
        stripped_post = remove_non_frequent(topn, sentence.strip().lower())
        #if len(stripped_post) > 20 and len(stripped_post) < 50 and stripped_post not in return_lines:
        if stripped_post not in return_lines:
            return_lines.append(stripped_post)
        #print(str(perc) + ": " + stripped_post)
        
    return return_lines

outfile = codecs.open('1506_only_top0.005_all.txt', mode='w')

# 20 top authors from 401 & 510
auth_text = get_by_probability([authors_410,authors_501], max_count, 20)
auth_io = get_topn_authors([authors_410,authors_501], 30)[-10:] # get_by_probability([authors_410,authors_501], 50, 30)[-10:]
#print(auth_text)
#print(auth_io)


pbar = ProgressBar(widget=[Percentage(), Bar()], maxval=max_count).start()

# 300 lines
index = 0
#lines = generate_lines_maxapp(max_count, 0.025)
lines = process_all(0.005)
print(len(lines))
for line in lines:
    l = ''.join(line)
    outfile.write(auth[index] + ': ' + l + '\n')
    #print(auth_text[index] + ': ' + l)
    #print(index)
    pbar.update(index)
    if random.uniform(0, 1) > 0.7:
        lj = ""
        if random.uniform(0, 1) > 0.5:
            lj = " LEFT THE ROOM"
        else:
            lj = " JOINED THE ROOM"
        #print(random.choice(auth_io)[0] + lj)
        outfile.write(random.choice(auth_io)[0] + lj + '\n')
    
    index += 1

pbar.update(max_count)
outfile.close()
# random in/outs for last 10
# top_n 2000 / 2500



  5% (6583 of 130000) |                  | Elapsed Time: 0:02:32 ETA:   0:47:39

114124


 86% (111899 of 130000) |#############   | Elapsed Time: 0:02:34 ETA:   0:00:00

In [351]:
# attempt to find similar sentences by matching vocabulary 

#li = ['her', 'her', 'her', 'he']
li = ['are', 'fake']
li = ['what', 'she', 'does']
#li = ['what', 'what', 'what']
li = ['is', 'is', 'is', 'is']
li = ['being'] * 3

#li = ['good', 'good', 'am']
#li = ['here', 'here', 'here']
#li = ['nice', 'nice', 'very']
#li = ['well', 'well']
#li = ['ll', 'll', 'll', 'll']
li = ['no'] * 4
#li = ['just', 'well', 'i']
index = 0
for post in all_posts:
    p = post[1]
    #print(p)
    success = True
    for l in li:
        if l in p:
            index = p.find(l)
            length = len(l)
            endindex = index + length
            subs = p[index:endindex] 
            leftover = p[:index] + p[endindex:]
            p = leftover
        else:
            success = False
            #print(post[1] + ' failed')
    if success and len(post[1]) < 80: 
        #print(len(post[1]))
        print(post[1])
    index += 1
        


no no no no
annnnoooooooorrrrrraaaaaaaaaaaaa
i know you knew, i just wanna know the interest of it, knowing that you know
we have snow nordicgoddess but not enough to be snowed in .... yet lol
no pwincess_aria no no no no
pretends to know nothing then no one expects me to know anything
no nononononono
nnnnnoooo more poucing, hehe
nnnnnooooooooooooooooooooooo
yeah i know dj`annora not a good thing right now
07,01 limits: 00 no pee and scat, no permanent injuries, no animals nor kids
lol no no no no
nnnnnnnnnnnooooooooooooo


In [115]:
# todo: 1406
# insert sentence into parser
# extract gramatical structure
# try to match through corpus
# use pre-processed text, with dashes 

import spacy
import random
from progressbar import ProgressBar, Bar, Percentage

# punctation is NFP or PUNCT

nlp = spacy.load('en_core_web_sm')

def get_tag_sequence(sen, detailed=False):
    doc = nlp(sen)
    sentence_pos_list = []
    sentence_tag_list = []

    for token in doc:
        sentence_pos_list.append(token.pos_)
        sentence_tag_list.append(token.tag_)
    
    if detailed:
        return sentence_tag_list
    
    return sentence_pos_list

def match_sequence(seq_to_match, seq_of_target):
    to_match = ''.join(seq_to_match)
    of_target = ''.join(seq_of_target)
    #print(to_match)
    #print(of_target)
    if to_match in of_target:
        return True
    return False

def match_sequences(sequences, seq_of_target):
    mmm = []
    for s in sequences:
        mmm.append(match_sequence(s, seq_of_target))
    #print(mmm)
    return mmm

    
input_file = open('1406_only_top0.025_all.txt', mode='r', encoding='utf8').readlines()

detailed = True    

ssss = []
ssss.append('---- and i was ----')
ssss.append('---- green and red ----')
ssss.append('---- such as ---- or ----')
ssss.append('---- hi ---- and ----')
ssss.append('---- of the wall ---- of the bank')
taglist = []
for s in ssss:
    taglist.append(get_tag_sequence(s, detailed))
    
print('input: ' + sentence)
matched_sentences = []
index = 0
bar = ProgressBar(widget=[Percentage(), Bar()], maxval=len(input_file)).start()
for line in input_file:
    if len(line) > 2:
        tseq = get_tag_sequence(line.rstrip(), detailed)
        matched = match_sequences(taglist, tseq)
        ii = 0
        for m in matched:
            if m:
                matched_sentences.append('matched pattern ' + str(ii) + ': ' + line.rstrip())
                print('matched pattern ' + str(ii) + ': ' + line.rstrip())
            else:
                pass
            ii += 1
        index += 1
        #if index > 2000:
        #    break
        bar.update(index)
for m in matched_sentences:
    print(m)
if len(matched_sentences) == 0:
    print('no results :(')

input: to be ---- to see


  0% (1281 of 164986) |                  | Elapsed Time: 0:00:15 ETA:   0:27:22

matched pattern 2: Athena: -------- i enjoy the open air ---------- such as ------ or ---------


  1% (3173 of 164986) |                  | Elapsed Time: 0:00:34 ETA:   0:26:48

matched pattern 1: Jothom: --- why im ------------ tired and body -----


...

matched pattern 0: +DJ`Mercury: so by the ----- of ---- --- and she was --


 27% (45377 of 164986) |####             | Elapsed Time: 0:07:50 ETA:   0:21:19

matched pattern 2: Silver_haired`Fox: damn i thought i was ---- thin at --- and ---


 99% (164985 of 164986) |############### | Elapsed Time: 0:29:25 ETA:   0:00:00

matched pattern 2: Athena: -------- i enjoy the open air ---------- such as ------ or ---------
matched pattern 1: Jothom: --- why im ------------ tired and body -----
matched pattern 0: +DJ`Mercury: so by the ----- of ---- --- and she was --
matched pattern 2: Silver_haired`Fox: damn i thought i was ---- thin at --- and ---


In [ ]:
import spacy
import random
from progressbar import ProgressBar, Bar, Percentage

nlp = spacy.load('en_core_web_sm')

possible = ['for', 'her', 'that', 'hello', 'on', 'me', 'hey', 'are', 'with', 'good', 'all', 'im', 'have', 'as', 'up', 'at', 'your', 
            'not', 'like', 'his', 'be', 'just', 'no', 'so', 'its', 'how', 'back', 'out', 'one', 'but', 'was', 'what', 'if', 'dont', 
            'well', 'here', 'or', 'she', 'we', 'do', 'nice', 'into']
input_file = open('1406_only_top0.025_all.txt', mode='r', encoding='utf8').readlines()
max_line_counter = 0
out_file = open('1406_min5_words_relative0.45.txt', 'w')
perc = 0.45


for line in input_file:
    counter = 0
    wordcount = len(line.split())
    
    for word in line.split():
        for p in possible:
            if word == p:
                counter += 1
    if counter/float(wordcount) > perc and wordcount > 10:
        out_file.write(line)
        print(line)
    max_line_counter += 1
    #if max_line_counter > 90000:
    #    break
    
out_file.close()

In [45]:
import spacy
import random
from progressbar import ProgressBar, Bar, Percentage

nlp = spacy.load('en_core_web_sm')

possible = ['for', 'her', 'that', 'hello', 'on', 'me', 'hey', 'are', 'with', 'good', 'all', 'im', 'have', 'as', 'up', 'at', 'your', 
            'not', 'like', 'his', 'be', 'just', 'no', 'so', 'its', 'how', 'back', 'out', 'one', 'but', 'was', 'what', 'if', 'dont', 
            'well', 'here', 'or', 'she', 'we', 'do', 'nice', 'into']

input_file = open('1406_only_top0.025_all.txt', mode='r', encoding='utf8').readlines()
max_line_counter = 0
out_file = open('1506_structure_02.txt', 'w')

perc = 0.3
chosen = []
wordcount_min = 10

for line in input_file:
    #perc += 0.002
    counter = 0
    #print(line)
    line = line.split(':')
    
    if len(line) < 2:
        continue
    auth = line[0]
    line = line[1]
    if len(line) < 3:
        continue
    wordcount = len(line.split())


    for word in line.split():
        for p in possible:
            if word == p:
                counter += 1

    if counter / float(wordcount) > perc and wordcount > wordcount_min:
        perc += 0.002
        chosen.append(line)
        out_file.write(auth + ': ' + line)
        print(str(perc) + ': ' + line.rstrip())
        
    if len(chosen) > 150:
        break
    #max_line_counter += 1
    #if max_line_counter > 90000:
    #    break
print(len(chosen))

out_file.write('--------------------------------------------------------------------------------\n')

short_chosen = []
min_len = 50
max_len = 140
for line in input_file:
    line = line.split(':')
    if len(line) < 2:
        continue 
    auth = line[0]
    line = line[1]
    if len(line) < 3:
        continue
    counter = 0
    wordcount = len(line.split())
    linelen = len(line)
    
    if wordcount == 0:
        continue
    
    for word in line.split():
        for p in possible:
            if word == p:
                counter += 1
                
    
    if counter / float(wordcount) > perc and linelen < max_len and linelen > min_len:
        min_len -= 1
        max_len -= 2
        perc += 0.002
        short_chosen.append(line)
        out_file.write(auth + ': ' + line)
        print(str(perc) + ': ' + line.rstrip())
        
    if len(short_chosen) > 70:
        break
    #max_line_counter += 1

print(len(short_chosen))
out_file.close()

0.302:  winks at ---------------- as he must be here for ---- -------- ---- by his wife
0.304:  grins from the ------ as her ass -------- back the ----- between her cheeks ------ with more panties as his ----- ---- deep between --------- as she ---- her hips to get -----------
0.306:  his ------- as he ------ at her right ------ and pushing
0.308:  a hand up into her ----- to feel that ---- ---- -------- her ------ as he sucks the back and left side of her neck
0.31:  ---- in her hair as that hole has a good ----
0.312:  as his finger ----- just a bit ------ now as she makes those
0.314:  slide into that hair once more and pull her back sliding once more to his knees
0.316:  coming as she ------- back down once more with her face and tits to the floor -------- that ------------ ass back up like she was made to do
0.318:  ---- and ------ out sitting back up -------- his ------ with a warm smile looking --------- over as she ----- off
0.32:  im --------- horny and not getting along as we

In [48]:
input_file = open('1406_only_top0.025_all.txt', mode='r', encoding='utf8').readlines()
import random

possible = ['for', 'her', 'that', 'hello', 'on', 'me', 'hey', 'are', 'with', 'good', 'all', 'im', 'have', 'as', 'up', 'at', 'your', 
            'not', 'like', 'his', 'be', 'just', 'no', 'so', 'its', 'how', 'back', 'out', 'one', 'but', 'was', 'what', 'if', 'dont', 
            'well', 'here', 'or', 'she', 'we', 'do', 'nice', 'into']

def checkline(line):
    line = line.split(':')
    if len(line) < 2:
        return False
    line = line[1]
    if len(line) < 2:
        return False
    return True

dofirst = True

finlines = []
perc = 0.3
wordlen_min = 60
if dofirst:
    for i in range(130):
        while True:
            line = random.choice(input_file)
            if not checkline(line):
                line = random.choice(input_file)
                continue   
            auth = line.split(':')[0]
            line = line.split(':')[1]

            counter = 0
            wordcount = len(line.split())

            if wordcount == 0:
                continue

            linelen = len(line)
            for word in line.split():
                for p in possible:
                    if word == p:
                        counter += 1

            if counter / float(wordcount) > perc and linelen > wordlen_min and line not in finlines: 
                print(line.rstrip())
                finlines.append(auth + ': ' + line)
                perc += 0.0015
                break

print('###########################################################################################################################')
        
wordlen_min = 60
wordlen_max = 70
for i in range(70):
    tried = 0
    while True and tried < 10000:
        tried += 1
        line = random.choice(input_file)
        if not checkline(line):
            line = random.choice(input_file)
            continue   
        auth = line.split(':')[0]
        line = line.split(':')[1]
            
        counter = 0
        wordcount = len(line.split())
        
        if wordcount == 0:
            continue
        
        linelen = len(line)
        for word in line.split():
            for p in possible:
                if word == p:
                    counter += 1
        
        if counter / float(wordcount) > perc and linelen > wordlen_min and linelen < wordlen_max and line not in finlines: 
            print(line.rstrip())
            finlines.append(auth + ': ' + line)
            perc += 0.0015
            wordlen_min -= 1
            wordlen_max -= 1
            break
print(len(finlines))

outfile = open('1506_structure_02_03.txt', 'w')
for line in finlines:
    outfile.write(line)
outfile.close()


 its --------- for you to open your mouth in a room like this
 but im doing it out of ------- because my ----- rather long for me
 not yet but its ------ up on another --- for later roughone
 --------------------------------------------- saffronwh how are you -----
 -------- and ------- are pretty ----- but i have not heard that any of the -------- have been closed
 i have - ---------- ---------- -------- who have ------- to work all week so im bored
 ------ her lips kissing him back just as innocent and sweet
 well fuck youre not a damn --- anymore man the fuck up and ask her out its really ------- all she can do is say no then you just move on
 i think we all ------ it was in the ----- its well over its ---------
 ------- her neck -------- it to ---- ------ her lip to ----- her ---- his soft kisses feels so nice as she ----- her legs ever so slightly
 chuckles at the ------ seyyal im not -------- just ------- the best for you
 ------ up onto a ------ ------ ------ on all ------ then 

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [11]:
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mrzl\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [43]:
# count i/s in 410/501
leaves410 = 0
entrances410 = 0
leaves501 = 0
entrances501 = 0

for line in codecs.open('1006/410_utf8.txt', mode='r', encoding='utf-8'):
    if 'has left' in line or 'has quit' in line:
        leaves410 += 1
    if 'joined the channel' in line:
        entrances410 += 1
    #if ':' in line and not in_blocklist(blocklist, line.strip()): # blocked?:
        #sentence = line[line.find(":"):]
        #all_posts.append((author, sentence.strip().lower()))
        #all_posts_410.append((author, sentence.strip().lower()))

        #author = line[:line.find(":")]
        #if author not in authors: # save things
            #authors[author] = set() # init list for author
        #if author not in authors_410:
            #authors_410[author] = set()
        #authors[author].add(sentence.strip().lower())
        #authors_410[author].add(sentence.strip().lower())


for s in codecs.open("1006/501_utf8.txt", mode='r', encoding='utf-8').readlines():
    if 'has left' in s or 'has quit' in s:
        leaves501 += 1
    if 'joined the channel' in s:
        entrances501 += 1
    #if ':' in s and 'has left the channel' not in s and 'has quit' not in s and 'joined the channel' not in s:
        #if not in_blocklist(blocklist, s.strip()): # blocked?
            #sentence = s[s.find(">")+2:]
            #all_posts.append((author, sentence.strip().lower()))
            #all_posts_501.append((author, sentence.strip().lower()))

            #author = s[s.find("<")+1:s.find(">")]
            #if author not in authors: # save things
            #    authors[author] = set() # init list for author
            #if author not in authors_501:
            #    authors_501[author] = set()
            #authors[author].add(sentence.strip().lower())
            #authors_501[author].add(sentence.strip().lower()) 
            
            
         
print(leaves410)
print(entrances410)
print(leaves501)
print(entrances501)

103
111
125
117


In [ ]:
from nltk.corpus import wordnet as wn
import random

def get_random_syn(word):
    r = word
    syns = wn.synsets(word)
    lemmas = []
    for ss in syns:
        for lem in ss.lemma_names():
            lemmas.append(lem)
            #r = lem
            #if lem is not word:
            #    return lem
    return lemmas

def replace_sentence(sentence):
    return_sentence = ""
    for word in sentence.split():
        lemmas = get_random_syn(word)
        if len(lemmas) > 0:
            #print(word + str(lemmas))
            r = random.choice(lemmas)
            if r is "atomic_number_53":
                r = 'I'
            return_sentence += r + " "
        else:
            return_sentence += word + ' '
    return return_sentence



for i in range(500):
    
    sen = random.choice(all_posts)
    if len(sen[1]) > 10:
        print('-------------------------')
        print(sen[1])
        print(replace_sentence(sen[1]))
        print(replace_sentence(sen[1]))
        print(replace_sentence(sen[1]))
        print(replace_sentence(sen[1]))
        print(replace_sentence(sen[1]))

In [ ]:
# CURVES generation

random.seed(1)
#render_plain('1306_removed_digits_entrances_01_ORIGINAL.txt', 500, 3860)
#render_by_osc('osc_config_500_late_slowexp.csv', '1306_cleaner_group1&2&3_01.txt', 3860)
print('finito')
#generate_lines_linear(20)

generate_lines_from(0.00, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.10, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.20, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.50, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.80, 0.0001, 10)

In [101]:
# saving word groups
lists = []
#lists.append((1, 5))
#lists.append((6, 20))
#lists.append((21, 50))
#lists.append((51, 100))
#lists.append((101, 500))
lists.append((3000, 10000))

outfile = codecs.open('3000_10000_groups.txt', 'w')

for l in lists:
    possible_words = []
    #_config = config[index]
    _min = l[0]
    _max = l[1]
    app_list = []
    for i in range(_min, _max+1):
        app_list.append(i)

    # all allowed words
    selected_appearances = get_by_appearance_list(app_list)
    outfile.write(' '.join(selected_appearances))
    #possible_words.append((selected_appearances, _config))
    
outfile.close()
    


In [ ]:
# postprocess act1&2

for sentence in all_posts_501:
    print(sentence[0] + " -> " + remove_non_frequent(get_topn_percent(0.01), sentence[1]))